In [27]:
import pandas as pd
import json
import numpy as np
import torch
from torch.autograd import Variable
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import seaborn as sns

In [28]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [29]:
with open('order_dict.json') as f:
    data = json.load(f)

In [30]:
order_dict = {int(k):pd.DataFrame(v).set_index('day_of_month') for k, v in data.items()}

In [31]:
order_dict[1]

,demand,day_of_week,order_hour_mode,order_hour_08,order_hour_95,att1_high,att1_miss,att2_high,att2_miss,type_1_percent,...,fast_promise,original_unit_price_mean,discount_rate_mean,direct_discount_rate_mean,quantity_discount_rate_mean,bundle_discount_mean,gift_mean,dc_ori_mode,dc_ori_num,cluster_id
day_of_month,,,,,,,,,,,,,,,,,,,,,
1,5,3,12,0.000000,1.000000,0.750000,0.250000,0.000000,0.750000,0.000000,...,0.250000,121.475000,0.070538,0.061069,0.009470,0.000000,0.000000,7,1,0
1,18,3,5,0.444444,0.555556,1.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,7,1,1
1,57,3,23,0.111111,0.400000,0.488889,0.222222,0.333333,0.355556,0.933333,...,0.888889,108.322222,0.468941,0.104816,0.238231,0.000000,0.044444,7,3,2
1,11,3,14,0.090909,0.636364,1.000000,0.000000,0.545455,0.090909,0.090909,...,0.272727,144.081818,0.669827,0.568953,0.027548,0.000000,0.000000,7,1,3
1,58,3,12,0.111111,0.416667,0.777778,0.083333,0.500000,0.111111,0.333333,...,0.472222,125.488889,0.421472,0.170375,0.178280,0.000000,0.000000,7,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,39,5,14,0.028571,0.742857,0.400000,0.342857,0.628571,0.057143,0.057143,...,0.171429,104.162857,0.335590,0.118868,0.003852,0.013378,0.028571,7,2,6
31,16,5,15,0.000000,0.866667,0.733333,0.200000,0.533333,0.200000,0.000000,...,0.000000,1.200000,0.933333,0.000000,0.000000,0.000000,0.933333,7,1,7
31,26,5,13,0.090909,0.636364,0.590909,0.272727,0.454545,0.272727,0.727273,...,0.727273,111.381818,0.412384,0.084126,NaN,0.039330,0.272727,1,3,8


In [32]:
df = order_dict[1].copy()
df = df.fillna(0)
df.replace(np.inf,0, inplace = True)

# only filter by sigfinicant features
# df = df[['demand','quantity_discount_rate_mean','cluster_id','dc_ori_mode','day_of_week']]
# df = df[['demand','quantity_discount_rate_mean']]
# df

# convert data type for categorical features
df['cluster_id'] = df['cluster_id'].astype('object')
df['dc_ori_mode'] = df['dc_ori_mode'].astype('object')
df['day_of_week'] = df['day_of_week'].astype('object')
# order_hour_mode??

cat = ['cluster_id','dc_ori_mode','day_of_week']
num = df.drop(cat+['demand'], axis = 1).columns
print(num)

Index(['order_hour_mode', 'order_hour_08', 'order_hour_95', 'att1_high',
       'att1_miss', 'att2_high', 'att2_miss', 'type_1_percent', 'no_promise',
       'fast_promise', 'original_unit_price_mean', 'discount_rate_mean',
       'direct_discount_rate_mean', 'quantity_discount_rate_mean',
       'bundle_discount_mean', 'gift_mean', 'dc_ori_num'],
      dtype='object')


In [33]:
# train test split
df_enc = pd.get_dummies(df, columns = cat, drop_first = True)
train_enc = df_enc.query('1<= day_of_month <= 24')
test_enc = df_enc.query('24 < day_of_month <= 31')

# train = df.query('1<= day_of_month <= 24')
# test = df.query('24 < day_of_month <= 31')

# train_enc = pd.get_dummies(train, columns = cat, drop_first = True)
# test_enc = pd.get_dummies(test, columns = cat)

# print(train_enc.dtypes)
# print(test_enc.dtypes)

# standardize data
scaler = StandardScaler()

# x_train = train_enc[['quantity_discount_rate_mean','day_of_week_3','cluster_id_6','cluster_id_7','cluster_id_8','cluster_id_10','dc_ori_num','dc_ori_mode_7']].to_numpy()
x_train = train_enc.drop(['demand', 'discount_rate_mean','type_1_percent'], axis = 1).to_numpy() # remove highest vif column
x_train = scaler.fit_transform(x_train)

y_train = train_enc['demand'].to_numpy().reshape(len(train_enc), 1)

# x_test = test_enc[['quantity_discount_rate_mean','day_of_week_3','cluster_id_6','cluster_id_7','cluster_id_8','cluster_id_10','dc_ori_num','dc_ori_mode_7']]
x_test = test_enc.drop(['demand', 'discount_rate_mean','type_1_percent'], axis = 1) 
x_test = scaler.fit_transform(x_test)

y_test = test_enc['demand'].to_numpy().reshape(len(test_enc), 1)

# print(list(zip(x_train, y_train)))
# torch.from_numpy(x_train)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(198, 35) (198, 1)
(57, 35) (57, 1)


In [34]:
def SPOLoss (d_pred, d_actual):
    # W: items to stock the warehouse
    # h: inventory cost
    # s: shipping cost
    # ci: capacity limit for warehouse i = 1000 (per month)
    cost = 0.0
    max_capacity = 1000.0 / 31
    for i in range(0, len(d_pred)):
        if d_pred[i] <= max_capacity:
            w = d_pred[i]
        else:
            w = max_capacity
        if w > d_actual[i]:
            h = 2.920 * (w - d_actual[i])  # average inventory cost per sku * (w - target)
            s = 0
        else:
            h = 0
            s = 0.803 * (d_actual[i] - w)
        cost += h + s
    return cost

def OSR2(y_train, y_test, y_pred):
    
    SSE = np.sum((y_test - y_pred)**2)
    SST = np.sum((y_test - np.mean(y_train))**2)
                 
    return (1 - SSE/SST)

def generate_scores(y_true, y_pred):
    MSE = mean_squared_error(y_true, y_pred)
    R2 = r2_score(y_true, y_pred)
    SPO = SPOLoss(y_pred, y_true)
    SPO_mean = SPO / len(y_pred)
    #print("Train MSE =", MSE)
    #print('Train R^2 =', R2) 
    #print('Train SPO Loss =', SPO)
    return pd.DataFrame({'MSE': MSE, 'R^2': R2, 'SPO Loss': SPO, 'SPO Loss Ave': SPO_mean})

def evaluation(model, x_train, x_test, y_train, y_test, plot = True):
    
    """
    this function only works for pytorch model
    
    return a 2x4 dataframe: 
        - one row for training set, the other row for testing
        - 4 columns are 4 scores: MSE, R2, SPO, SPO Ave
    """

    pred_y_train = model.predict(x_train)
    pred_y_test = model.predict(x_test)

    score_df = pd.concat([generate_scores(y_train, pred_y_train), generate_scores(y_test, pred_y_test)])
    score_df.index = ['train', 'test']
    
    return score_df

### OLS

In [35]:
ols = LinearRegression().fit(x_train, y_train)
evaluation(ols, x_train, x_test, y_train, y_test)

,MSE,R^2,SPO Loss,SPO Loss Ave
train,56.959344,0.768150,2054.963949,10.378606
test,77.052244,0.524061,729.699173,12.801740


### Ridge Regression

In [36]:
alpha_grid = {'alpha': np.logspace(-1, 5, num=50, base=10)}

rr = Ridge(random_state=88)
rr_cv = GridSearchCV(rr, alpha_grid, scoring='neg_mean_squared_error', cv=10)
rr_cv.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=Ridge(random_state=88),
             param_grid={'alpha': array([1.00000000e-01, 1.32571137e-01, 1.75751062e-01, 2.32995181e-01,
       3.08884360e-01, 4.09491506e-01, 5.42867544e-01, 7.19685673e-01,
       9.54095476e-01, 1.26485522e+00, 1.67683294e+00, 2.22299648e+00,
       2.94705170e+00, 3.90693994e+00, 5.17947468e+00, 6.86648845e+00,
       9.10298178e+00, 1.20679264e+01, 1...
       2.68269580e+02, 3.55648031e+02, 4.71486636e+02, 6.25055193e+02,
       8.28642773e+02, 1.09854114e+03, 1.45634848e+03, 1.93069773e+03,
       2.55954792e+03, 3.39322177e+03, 4.49843267e+03, 5.96362332e+03,
       7.90604321e+03, 1.04811313e+04, 1.38949549e+04, 1.84206997e+04,
       2.44205309e+04, 3.23745754e+04, 4.29193426e+04, 5.68986603e+04,
       7.54312006e+04, 1.00000000e+05])},
             scoring='neg_mean_squared_error')

In [37]:
evaluation(rr_cv, x_train, x_test, y_train, y_test)

,MSE,R^2,SPO Loss,SPO Loss Ave
train,59.887011,0.756233,2052.545092,10.366389
test,61.541008,0.619872,616.681282,10.818970


### Lasso Regression

In [38]:
lasso = Lasso()
alpha_grid = {'alpha': np.logspace(-5, 3, num=100, base=10)}

lasso_cv = GridSearchCV(lasso, alpha_grid, scoring='neg_mean_squared_error', cv=10)
lasso_cv.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=Lasso(),
             param_grid={'alpha': array([1.00000000e-05, 1.20450354e-05, 1.45082878e-05, 1.74752840e-05,
       2.10490414e-05, 2.53536449e-05, 3.05385551e-05, 3.67837977e-05,
       4.43062146e-05, 5.33669923e-05, 6.42807312e-05, 7.74263683e-05,
       9.32603347e-05, 1.12332403e-04, 1.35304777e-04, 1.62975083e-04,
       1.96304065e-04, 2.36448941e-04, 2.84803587e-04, 3...
       1.38488637e+01, 1.66810054e+01, 2.00923300e+01, 2.42012826e+01,
       2.91505306e+01, 3.51119173e+01, 4.22924287e+01, 5.09413801e+01,
       6.13590727e+01, 7.39072203e+01, 8.90215085e+01, 1.07226722e+02,
       1.29154967e+02, 1.55567614e+02, 1.87381742e+02, 2.25701972e+02,
       2.71858824e+02, 3.27454916e+02, 3.94420606e+02, 4.75081016e+02,
       5.72236766e+02, 6.89261210e+02, 8.30217568e+02, 1.00000000e+03])},
             scoring='neg_mean_squared_error')

In [39]:
evaluation(lasso_cv, x_train, x_test, y_train, y_test)

,MSE,R^2,SPO Loss,SPO Loss Ave
train,58.451712,0.762075,2043.207332,10.319229
test,64.134694,0.603851,630.143937,11.055157


In [40]:
# compile results for models trained using sklearn
sk_result = []

for model, name in zip([ols, rr_cv, lasso_cv],['ols','ridge','lasso']):
    score_df = evaluation(model, x_train, x_test, y_train, y_test, plot = False)
    sk_result.append(pd.concat({name:score_df}))
    
pd.concat([sk_result[0], sk_result[1], sk_result[2]])

MSE       R^2     SPO Loss  SPO Loss Ave
ols   train  56.959344  0.768150  2054.963949     10.378606
      test   77.052244  0.524061   729.699173     12.801740
ridge train  59.887011  0.756233  2052.545092     10.366389
      test   61.541008  0.619872   616.681282     10.818970
lasso train  58.451712  0.762075  2043.207332     10.319229
      test   64.134694  0.603851   630.143937     11.055157

In [25]:
comparison_data = {
    'Ridge Regression': ['{:.3f}'.format(OSR2(y_train, y_test, rr_cv.predict(x_test))),
                         '{:.4f}'.format(RMSE(y_test, rr_cv.predict(x_test))),
                         '{:.3f}'.format(MAE(y_test, rr_cv.predict(x_test)))],
    
#     'Random Forest Regression': ['{:.3f}'.format(OSR2(y_train, y_test, rf_cv.predict(X_test_rf))),
#                                  '{:.4f}'.format(RMSE(y_test, rf_cv.predict(X_test_rf))),
#                                  '{:.3f}'.format(MAE(y_test, rf_cv.predict(X_test_rf)))]
 }

comparison_table = pd.DataFrame(data=comparison_data, index=['OSR2', 'Out-of-sample RMSE', 'Out-of-sample MAE'])
comparison_table.style.set_properties(**{'font-size': '12pt',}).set_table_styles([{'selector': 'th', 'props': [('font-size', '10pt')]}])

,Ridge Regression
OSR2,0.620
Out-of-sample RMSE,7.8448
Out-of-sample MAE,6.077
